In [ ]:
# =====================================================================================
# Please note that the datasets used in this Notebook are not supplied due to privacy.
# The cleaning pipeline and dataset observation logs, however, are available. 
# The code can be repurposed with your own dataset considering its specifics.
# =====================================================================================

In [ ]:
# =====================================================================================
# --- Innitiation ---
# =====================================================================================

import pandas as pd
import warnings
import itertools
import string

# --- Global Configuration ---
# Suppress simple warnings for a cleaner output.
warnings.simplefilter("ignore")

# --- Constants ---
# String that indicates a non-informative observation.
OBSERVATION_STRING: str = "Nenhuma observação fornecida"

# --- File Paths ---
# important note: the input file used in this study has been preliminary cleaned and the purpose of this script is to prepare it for specific tasks
INPUT_FILE: str = 'logs_precleaned.csv'
OUTPUT_FILE: str = 'all_logs.csv'
GLOSSARY_OUTPUT_FILE: str = 'farm_id_glossary.csv'

In [ ]:
# =====================================================================================
# --- Helper Functions ---
# =====================================================================================
def preprocess_maintenance_logs(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans, filters, and prepares maintenance log data for analysis.

    This function performs the following sequential steps:
    1.  Removes a predefined list of columns and any fully empty columns.
    2.  Filters records to keep only those where 'Age at Event' is >= 1.
    3.  Removes infrastructure logs (where 'Turbine No' is null).
    4.  Filters out logs with missing component names.
    5.  Renames specific rotor blade components for granular analysis.
    6.  Cleans text fields ('Description', 'Observations') using regex.
    7.  Removes logs deemed uninformative based on text content.
    8.  Converts 'Event Date' to datetime objects and sorts the data.

    Args:
        df: The raw pandas DataFrame of maintenance logs.

    Returns:
        A fully preprocessed and sorted pandas DataFrame.
    """
    print("--- 2. Preprocessing Data ---")
    df_copy = df.copy()

    # 1. Remove specified and fully empty columns.
    cols_to_drop = ['Farm Number', 'Description_EN', 'Observations_EN']
    df_copy.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    df_copy.dropna(axis='columns', how='all', inplace=True)
    print("Removed specified and fully empty columns.")

    # 2. Filter out rows with invalid 'Age at Event' (< 1).
    initial_rows = len(df_copy)
    # Ensure the column is numeric, converting errors to NaN, then drop NaNs.
    df_copy['Age at Event'] = pd.to_numeric(df_copy['Age at Event'], errors='coerce')
    df_copy.dropna(subset=['Age at Event'], inplace=True)
    df_copy = df_copy[df_copy['Age at Event'] >= 1]
    print(f"Removed {initial_rows - len(df_copy)} rows with invalid 'Age at Event' (< 1).")

    # 3. Remove infrastructure logs.
    initial_rows = len(df_copy)
    df_copy.dropna(subset=['Turbine No'], inplace=True)
    print(f"Removed {initial_rows - len(df_copy)} infrastructure logs.")

    # 4. Filter out entries with no component name.
    df_copy.dropna(subset=['Component Name'], inplace=True)
    df_copy = df_copy[df_copy['Component Name'] != '']

    # 5. Rename rotor blade components.
    component_name_map = {
        'Rotor Blade System 1': 'Rotor Blade No. 1',
        'Rotor Blade System 2': 'Rotor Blade No. 2',
        'Rotor Blade System 3': 'Rotor Blade No. 3',
    }
    df_copy['Component Name'] = df_copy['Component Name'].replace(component_name_map)
    df_copy = df_copy[df_copy['Component Name'] != 'Rotor Blades Overall']
    print("Standardised component names.")

    # 6. Clean text fields.
    df_copy[['Description', 'Observations']] = df_copy[['Description', 'Observations']].fillna('')
    wo_pattern = r'\(WO\s\d+\)'
    log_code_pattern = r'\b\d{2}\.\d{3}\.\d{2}-\d{4}\b'
    multiple_spaces_pattern = r'\s+'
    for col in ['Description', 'Observations']:
        df_copy[col] = (df_copy[col].astype(str)
                        .str.replace(wo_pattern, '', regex=True)
                        .str.replace(log_code_pattern, '', regex=True)
                        .str.replace(multiple_spaces_pattern, ' ', regex=True)
                        .str.strip())
    
    # 7. Remove uninformative logs.
    initial_rows = len(df_copy)
    empty_both = (df_copy['Description'] == '') & (df_copy['Observations'] == '')
    short_desc = (df_copy['Description'].str.split().str.len() <= 1)
    uninformative_obs = df_copy['Observations'].isin(['', OBSERVATION_STRING])
    df_copy = df_copy[~(empty_both | (short_desc & uninformative_obs))]
    print(f"Removed {initial_rows - len(df_copy)} uninformative logs.")

    # 8. Convert to datetime and sort.
    df_copy['Event Date'] = pd.to_datetime(df_copy['Event Date'])
    df_copy.sort_values(by='Event Date', inplace=True, ignore_index=True)
    print("Sorted data by 'Event Date'.")

    print("\nData preprocessing complete.")
    return df_copy


def anonymise_farm_ids(
    df: pd.DataFrame
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Anonymises 'Farm ID' using two-letter codes and creates a glossary.

    Args:
        df: The preprocessed DataFrame.

    Returns:
        A tuple containing:
        - The DataFrame with the new anonymised 'Wind Farm' column.
        - A new DataFrame serving as a glossary to trace original IDs.
    """
    print("\n--- 3. Anonymising Farm IDs ---")
    
    # Generate two-letter codes (AA, AB, ..., ZZ).
    letters = string.ascii_uppercase
    two_letter_codes_iter = itertools.product(letters, repeat=2)
    anonymised_codes = ["".join(code) for code in two_letter_codes_iter]
    
    unique_farms = df['Farm ID'].unique()
    
    if len(unique_farms) > len(anonymised_codes):
        raise ValueError(
            f"Error: {len(unique_farms)} unique farms found, but only "
            f"{len(anonymised_codes)} unique two-letter codes are available."
        )

    # Create the mapping from unique Farm ID to an anonymised name.
    glossary = {
        farm_id: f'Wind Farm {anonymised_codes[i]}'
        for i, farm_id in enumerate(unique_farms)
    }
    
    # Map the anonymised names to a new 'Wind Farm' column.
    df['Wind Farm'] = df['Farm ID'].map(glossary)
    
    # Reorder columns to place 'Wind Farm' next to 'Farm ID'.
    try:
        farm_id_loc = df.columns.get_loc('Farm ID')
        cols = list(df.columns)
        cols.insert(farm_id_loc + 1, cols.pop(cols.index('Wind Farm')))
        df = df[cols]
    except KeyError:
        print("Warning: 'Farm ID' column not found. Cannot reorder.")

    # Create the glossary DataFrame.
    glossary_df = pd.DataFrame(
        list(glossary.items()), columns=['Farm ID', 'Anonymised ID']
    )
    
    print("Anonymisation complete and glossary created.")
    return df, glossary_df

In [4]:
# =====================================================================================
# --- Main Execution Pipeline ---
# =====================================================================================

# 1. Load Data
print("--- 1. Loading Data ---")
try:
    df_raw = pd.read_csv(INPUT_FILE)
    print(f"Successfully loaded. Initial shape: {df_raw.shape}\n")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{INPUT_FILE}'.")
    df_raw = None

if df_raw is not None:
    # 2. Run the full preprocessing pipeline
    df_processed = preprocess_maintenance_logs(df_raw)

    # 3. Anonymise farm IDs and create the glossary
    df_final, df_glossary = anonymise_farm_ids(df_processed)

    # 4. Save the final outputs
    print("\n--- 4. Saving Outputs ---")
    df_final.to_csv(OUTPUT_FILE, index=False)
    df_glossary.to_csv(GLOSSARY_OUTPUT_FILE, index=False)
    
    print(f"Final dataset shape: {df_final.shape}")
    print(f"Saved the analysis-ready dataset to '{OUTPUT_FILE}'.")
    print(f"Saved the farm ID glossary to '{GLOSSARY_OUTPUT_FILE}'.")
    
    print("\nProcess completed successfully. ✅")

--- 1. Loading Data ---
Successfully loaded. Initial shape: (12152, 13)

--- 2. Preprocessing Data ---
Removed specified and fully empty columns.
Removed 3 rows with invalid 'Age at Event' (< 1).
Removed 473 infrastructure logs.
Standardised component names.
Removed 4 uninformative logs.
Sorted data by 'Event Date'.

Data preprocessing complete.

--- 3. Anonymising Farm IDs ---
Anonymisation complete and glossary created.

--- 4. Saving Outputs ---
Final dataset shape: (10926, 11)
Saved the analysis-ready dataset to 'all_logs.csv'.
Saved the farm ID glossary to 'farm_id_glossary.csv'.

Process completed successfully. ✅


In [ ]:
# =====================================================================================
# --- Overview of the Data ---
# =====================================================================================

import pandas as pd
import numpy as np

# --- 1. Load Required Data ---
try:
    df_final = pd.read_csv('all_logs.csv')
    df_glossary = pd.read_csv('farm_id_glossary.csv')
    df_portfolio = pd.read_csv('portfolio.csv')
    print("✅ All necessary data files loaded successfully.\n")

except FileNotFoundError as e:
    print(f"❌ Error: Could not find a required file. Please check file paths.")
    print(f"Details: {e}")
    df_final = None

if df_final is not None:
    # Ensure 'Event Date' is in datetime format for calculations.
    df_final['Event Date'] = pd.to_datetime(df_final['Event Date'])

    # --- 2. Top 10 Most Frequent Component Names (with Rotor Blades aggregated) ---
    print("## Top 10 Most Frequently Occurring Component Names")
    
    # Temporarily group all 'Rotor Blade No. X' into a single 'Rotor Blades' category for this analysis.
    component_names_grouped = df_final['Component Name'].str.replace(r'Rotor Blade No. \d', 'Rotor Blades', regex=True)
    top_10_components = component_names_grouped.value_counts().nlargest(10)
    
    print(top_10_components)
    print("\n" + "-"*50 + "\n")


    # --- 3. Top 10 Most Frequent Wind Farms (Normalised) ---
    print("## Top 10 Most Frequent Wind Farms (Normalised)")
    
    # Calculate the monitoring period in years based on the log data.
    min_date = df_final['Event Date'].min()
    max_date = df_final['Event Date'].max()
    monitoring_period_years = (max_date - min_date).days / 365.25
    print(f"Monitoring Period: {monitoring_period_years:.2f} years (from {min_date.date()} to {max_date.date()})")

    # Prepare portfolio data using the 'TN' column.
    df_portfolio['Num Turbines'] = df_portfolio['TN'].astype(int)
    df_portfolio.rename(columns={'Code': 'Farm ID'}, inplace=True)
    
    # Merge portfolio and glossary to link anonymised names with turbine counts.
    farm_info = pd.merge(df_portfolio[['Farm ID', 'Num Turbines']], df_glossary, on='Farm ID')
    
    # Count maintenance events for each anonymised farm name.
    farm_event_counts = df_final['Wind Farm'].value_counts().reset_index()
    farm_event_counts.columns = ['Anonymised ID', 'Event Count']
    
    # Merge event counts with the farm information.
    farm_stats = pd.merge(farm_event_counts, farm_info, on='Anonymised ID')
    
    # Calculate the normalised failure frequency.
    farm_stats['Normalised Frequency (Events/Turbine/Year)'] = \
        (farm_stats['Event Count'] / farm_stats['Num Turbines']) / monitoring_period_years
        
    # Display the top 10.
    top_2_farms = farm_stats.sort_values(
        by='Normalised Frequency (Events/Turbine/Year)', ascending=False
    ).head(10)
    print(top_2_farms[['Anonymised ID', 'Normalised Frequency (Events/Turbine/Year)']])
    print("\n" + "-"*50 + "\n")
    

    # --- 4. Top 5 Most Frequent Turbines (with Unique IDs, Normalised) ---
    print("## Top 5 Most Frequently Failing Turbines (Normalised by Monitoring Period)")
    
    # Create a unique ID for each turbine by combining the farm's two-letter code and the turbine number.
    farm_code = df_final['Wind Farm'].str.split().str[-1]
    df_final['Unique Turbine ID'] = farm_code + '-' + df_final['Turbine No']
    
    # Count events per unique turbine ID.
    turbine_event_counts = df_final['Unique Turbine ID'].value_counts()
    
    # Normalise by the monitoring period.
    normalised_turbine_freq = turbine_event_counts / monitoring_period_years
    
    # Display the top 5.
    top_5_turbines = normalised_turbine_freq.nlargest(5).reset_index()
    top_5_turbines.columns = ['Unique Turbine ID', 'Normalised Frequency (Events/Year)']
    print(top_5_turbines)

    # --- 5. Wind Farms with the Highest Number of Logs (Raw Count) ---
    print("## Wind Farms with the Highest Number of Maintenance Logs (Raw Count)")

    # Check if the farm_stats DataFrame exists from the previous cell.
    if 'farm_stats' in locals() and isinstance(farm_stats, pd.DataFrame):
        
        # Sort the DataFrame by the 'Event Count' column in descending order.
        top_farms_by_raw_count = farm_stats.sort_values(
            by='Event Count', ascending=False
        ).reset_index(drop=True)
        
        # Select and display the relevant columns.
        print(top_farms_by_raw_count[['Anonymised ID', 'Event Count', 'Num Turbines']])

    else:
        print("❌ Error: The 'farm_stats' DataFrame was not found. Please run the previous cell first.")

✅ All necessary data files loaded successfully.

## Top 10 Most Frequently Occurring Component Names
Component Name
Rotor Blades                     1817
Converter System Generator 1     1065
Tower System Bottom Section       800
Central Hydraulic System          693
Turbine Control System            681
Generator 1                       627
Rotor Hub Unit                    565
Tower System Overall              552
MV-Transformation Generator 1     445
Gearbox                           412
Name: count, dtype: int64

--------------------------------------------------

## Top 10 Most Frequent Wind Farms (Normalised)
Monitoring Period: 18.57 years (from 2006-01-01 to 2024-07-28)
   Anonymised ID  Normalised Frequency (Events/Turbine/Year)
14  Wind Farm AP                                    4.325778
6   Wind Farm BB                                    3.607806
0   Wind Farm AN                                    2.977641
21  Wind Farm AW                                    2.943683
10  Wind 

In [ ]:
# =====================================================================================
# --- Creating Subsets for Specific Tasks ---
# =====================================================================================

# --- Subset by Component Name ---

target_component = 'Converter System Generator 1'

# Define the columns to keep for this subset.
columns_to_keep = [
    'Wind Farm', 
    'Unique Turbine ID', 
    'Description', 
    'Observations', 
    'Age at Event'
]

# Create the subset
# Note: This assumes 'Unique Turbine ID' was created in the previous step.
# If not, we create it here to be safe.
if 'Unique Turbine ID' not in df_final.columns:
    farm_code = df_final['Wind Farm'].str.split().str[-1]
    df_final['Unique Turbine ID'] = farm_code + '-' + df_final['Turbine No']
    
df_component_subset = df_final[df_final['Component Name'] == target_component][columns_to_keep]

print(f"✅ Created subset for component: '{target_component}'")
print(f"Shape of the new subset: {df_component_subset.shape}")
df_component_subset.to_csv('converter_subset.csv', index=False)

✅ Created subset for component: 'Converter System Generator 1'
Shape of the new subset: (1065, 5)


In [ ]:
# --- Subsets for Two Specified Wind Farms ---

# The names should be in the 'Wind Farm XX' format.
target_farm_1 = 'Wind Farm AO'
target_farm_2 = 'Wind Farm AM'

# Define the columns to keep for these subsets.
columns_to_keep = [
    'Component Name', 
    'Description', 
    'Observations'
]

# Create the first subset.
df_farm_1_subset = df_final[df_final['Wind Farm'] == target_farm_1][columns_to_keep]

# Create the second subset.
df_farm_2_subset = df_final[df_final['Wind Farm'] == target_farm_2][columns_to_keep]


print(f"✅ Created subset for farm '{target_farm_1}' ({df_farm_1_subset.shape[0]} rows)")
df_farm_1_subset.to_csv('farm_AO_subset.csv', index=False)

print(f"\n✅ Created subset for farm '{target_farm_2}' ({df_farm_2_subset.shape[0]} rows)")
df_farm_2_subset.to_csv('farm_AM_subset.csv', index=False)

✅ Created subset for farm 'Wind Farm AO' (697 rows)

✅ Created subset for farm 'Wind Farm AM' (745 rows)


In [ ]:
# --- Subset by Unique Turbine ID ---

# The ID must be in the 'XX-WT-NNN' format.
target_turbine_id = 'AP-WT-003'

# Define the columns to keep for this subset.
columns_to_keep = [
    'WONUM',
    'Component Name',
    'Description',
    'Observations',
    'Age at Event',
    'Event Date'
]

# Create the subset
# Note: This assumes 'Unique Turbine ID' was created in a previous step.
if 'Unique Turbine ID' not in df_final.columns:
    farm_code = df_final['Wind Farm'].str.split().str[-1]
    df_final['Unique Turbine ID'] = farm_code + '-' + df_final['Turbine No']
    
df_turbine_subset = df_final[df_final['Unique Turbine ID'] == target_turbine_id][columns_to_keep]

print(f"✅ Created subset for turbine: '{target_turbine_id}'")
print(f"Shape of the new subset: {df_turbine_subset.shape}")
df_turbine_subset.to_csv('turbine_AP-WT-003_subset.csv', index=False)

✅ Created subset for turbine: 'AP-WT-003'
Shape of the new subset: (92, 6)


In [ ]:
# --- Create a Reproducible Random 25% Subset ---

# The `random_state` ensures that the random sample is always the same.
subset_fraction = 0.20
reproducibility_seed = 42

# Create the random subset using the .sample() method.
df_random_subset = df_final.sample(
    frac=subset_fraction, 
    random_state=reproducibility_seed
)

print(f"✅ Created a reproducible random subset with {subset_fraction * 100}% of the data.")
print(f"Original dataset shape: {df_final.shape}")
print(f"New random subset shape: {df_random_subset.shape}")
df_random_subset.to_csv('logs_subset.csv', index=False)

✅ Created a reproducible random subset with 20.0% of the data.
Original dataset shape: (10926, 12)
New random subset shape: (2185, 12)
